#Import necessary libraries



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.preprocessing import PolynomialFeatures

import joblib
from sklearn.metrics import r2_score, mean_squared_error
import statsmodels.api as sm
#  (high-level, simple to use)
import plotly.express as px
# (low-level, highly customizable)
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc

#  Load the dataset

In [ ]:
# Download the dataset
data_path = "https://storage.googleapis.com/edulabs-public-datasets/heart_disease_uci.csv"

# Load the dataset into a Pandas DataFrame
df = pd.read_csv(data_path)


In [ ]:
df

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,916,54,Female,VA Long Beach,asymptomatic,127.0,333.0,True,st-t abnormality,154.0,False,0.0,NaN,NaN,NaN,1
916,917,62,Male,VA Long Beach,typical angina,NaN,139.0,False,st-t abnormality,NaN,NaN,NaN,NaN,NaN,NaN,0
917,918,55,Male,VA Long Beach,asymptomatic,122.0,223.0,True,st-t abnormality,100.0,False,0.0,NaN,NaN,fixed defect,2
918,919,58,Male,VA Long Beach,asymptomatic,NaN,385.0,True,lv hypertrophy,NaN,NaN,NaN,NaN,NaN,NaN,0


**Column description**

- id: unique id
- age: age in years
- sex: gender
- dataset: location of data collection
- cp: chest pain type
- trestbps: resting blood pressure
- chol: cholesterol measure
- fbs: fasting blood sugar
- restecg: ecg observation at resting condition
- thalch: maximum heart rate achieved
- exang: exercise induced angina
- oldpeak: ST depression induced by exercise relative to rest
- slope: the slope of the peak exercise ST segment
- ca: number of major vessels (0-3) colored by flourosopy
- thal: thal
- num: target [0=no heart disease; 1,2,3,4 = stages of heart disease ]

# Your task

- In this task you are going to train improved (hopefully) Logistic Regression model to detect whether there is heart disease.

1. Review all the missing values, find one numeric feature that is most important to impute - for this one you will use Linear Regression for imputation task.

2. Impute the rest of the features with basic methods - mean / median / mode by group

3. Train Linear Regression for imputation purposes for the feature you selected in (1.)

4. Review all the features and see whehter there is a need in feature engineering - check both whether there is a need to perform transformations on numeric features and if there is a need to "combine" multiple categorical / numerical features.

5. Train Logistic regression

6. Review the performance of the model with different metrics

7. Find the best threshold

8. Send me your solution :)

In [ ]:
df['is_heart_disease'] = df['num'].apply(lambda x: 1 if x > 0 else 0)
df['is_heart_disease'] = df['is_heart_disease'].astype('bool')
df.drop(columns=['num'], inplace=True)

In [ ]:
df['is_male'] = df['sex'].map({'Male': True, 'Female': False})
df.drop(columns=['sex'], inplace=True)

In [ ]:
df['age'] = df['age'].astype('int32')

In [ ]:
df['slope'] = df['slope'].fillna('no_data')
df['thal'] = df['thal'].fillna('no_data')
df.drop(columns=['dataset'], inplace=True)
df.drop(columns=['id'], inplace=True)

In [ ]:
df["chol"] = df["chol"].replace(0, np.nan)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               920 non-null    int32  
 1   cp                920 non-null    object 
 2   trestbps          861 non-null    float64
 3   chol              718 non-null    float64
 4   fbs               830 non-null    object 
 5   restecg           918 non-null    object 
 6   thalch            865 non-null    float64
 7   exang             865 non-null    object 
 8   oldpeak           858 non-null    float64
 9   slope             920 non-null    object 
 10  ca                309 non-null    float64
 11  thal              920 non-null    object 
 12  is_heart_disease  920 non-null    bool   
 13  is_male           920 non-null    bool   
dtypes: bool(2), float64(5), int32(1), object(6)
memory usage: 84.6+ KB


In [ ]:
df.isnull().sum()

,0
age,0
cp,0
trestbps,59
chol,202
fbs,90
restecg,2
thalch,55
exang,55
oldpeak,62
slope,0


In [ ]:
pd.crosstab(df['trestbps'], df['is_heart_disease'],normalize=True)

is_heart_disease,False,True
trestbps,,
0.0,0.000000,0.001161
80.0,0.001161,0.000000
92.0,0.000000,0.001161
94.0,0.002323,0.000000
95.0,0.000000,0.006969
...,...,...
180.0,0.006969,0.006969
185.0,0.000000,0.001161
190.0,0.001161,0.001161


In [ ]:
df['trestbps']=df['trestbps'].fillna(df['trestbps'].mean())

In [ ]:
pd.crosstab(df['chol'], df['is_heart_disease'],normalize=True)

is_heart_disease,False,True
chol,,
85.0,0.001393,0.000000
100.0,0.001393,0.001393
117.0,0.000000,0.001393
126.0,0.001393,0.000000
129.0,0.001393,0.000000
...,...,...
491.0,0.000000,0.001393
518.0,0.000000,0.001393
529.0,0.000000,0.001393


In [ ]:
# df['chol']=df['chol'].fillna(df['chol'].mode()[0])

In [ ]:
pd.crosstab(df['restecg'], df['is_heart_disease'],normalize=True)

is_heart_disease,False,True
restecg,,
lv hypertrophy,0.089325,0.115468
normal,0.291939,0.308279
st-t abnormality,0.066449,0.128540


In [ ]:
df['restecg'] = df['restecg'].fillna(df['restecg'].mode()[0])

In [ ]:
pd.crosstab(df['thalch'], df['is_heart_disease'],normalize=True)

is_heart_disease,False,True
thalch,,
60.0,0.000000,0.001156
63.0,0.000000,0.001156
67.0,0.000000,0.001156
69.0,0.001156,0.000000
70.0,0.000000,0.001156
...,...,...
190.0,0.002312,0.000000
192.0,0.001156,0.000000
194.0,0.001156,0.000000


In [ ]:
df['thalch'] = df['thalch'].fillna(df['thalch'].mean())

In [ ]:
pd.crosstab(df['oldpeak'], df['is_heart_disease'],normalize=True)

is_heart_disease,False,True
oldpeak,,
-2.6,0.000000,0.001166
-2.0,0.000000,0.001166
-1.5,0.000000,0.001166
-1.1,0.001166,0.000000
-1.0,0.000000,0.002331
-0.9,0.000000,0.001166
-0.8,0.000000,0.001166
-0.7,0.000000,0.001166
-0.5,0.001166,0.001166


In [ ]:
df['oldpeak'] = df['oldpeak'].fillna(df['oldpeak'].mean())

In [ ]:
pd.crosstab(df['ca'], df['is_heart_disease'],normalize=True)

is_heart_disease,False,True
ca,,
0.0,0.430421,0.155340
1.0,0.067961,0.148867
2.0,0.025890,0.106796
3.0,0.009709,0.055016


In [ ]:
df['ca'] = df['ca'].fillna(df['ca'].mode()[0])

In [ ]:
pd.crosstab(df['exang'], df['is_heart_disease'],normalize=True)

is_heart_disease,False,True
exang,,
False,0.388439,0.221965
True,0.063584,0.326012


In [ ]:
df['exang'] = df['exang'].fillna(df['exang'].mode()[0]).astype('bool')

<ipython-input-1211-f8fe18ebe7da>:1: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [ ]:
pd.crosstab(df['fbs'], df['is_heart_disease'],normalize=True)

is_heart_disease,False,True
fbs,,
False,0.425301,0.408434
True,0.053012,0.113253


In [ ]:
df['fbs'] = df['fbs'].fillna(df['fbs'].mode()[0]).astype('bool')

<ipython-input-1213-30396aa3036d>:1: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               920 non-null    int32  
 1   cp                920 non-null    object 
 2   trestbps          920 non-null    float64
 3   chol              718 non-null    float64
 4   fbs               920 non-null    bool   
 5   restecg           920 non-null    object 
 6   thalch            920 non-null    float64
 7   exang             920 non-null    bool   
 8   oldpeak           920 non-null    float64
 9   slope             920 non-null    object 
 10  ca                920 non-null    float64
 11  thal              920 non-null    object 
 12  is_heart_disease  920 non-null    bool   
 13  is_male           920 non-null    bool   
dtypes: bool(4), float64(5), int32(1), object(4)
memory usage: 72.0+ KB


In [ ]:
df.isnull().sum()

,0
age,0
cp,0
trestbps,0
chol,202
fbs,0
restecg,0
thalch,0
exang,0
oldpeak,0
slope,0


# Train LinearRegression to predict chol

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               920 non-null    int32  
 1   cp                920 non-null    object 
 2   trestbps          920 non-null    float64
 3   chol              718 non-null    float64
 4   fbs               920 non-null    bool   
 5   restecg           920 non-null    object 
 6   thalch            920 non-null    float64
 7   exang             920 non-null    bool   
 8   oldpeak           920 non-null    float64
 9   slope             920 non-null    object 
 10  ca                920 non-null    float64
 11  thal              920 non-null    object 
 12  is_heart_disease  920 non-null    bool   
 13  is_male           920 non-null    bool   
dtypes: bool(4), float64(5), int32(1), object(4)
memory usage: 72.0+ KB


In [ ]:
feature_selection = SelectKBest(score_func=f_regression, k=20)
preprocessor = ColumnTransformer(
    transformers=[
        (
            'numerical',
            'passthrough',
            ['age', 'trestbps',  'thalch', 'oldpeak']
        ),
        (
            'boolean',
            'passthrough',
            ['is_male', 'is_heart_disease', 'exang','fbs']
        ),
        (
            'one-hot-cat',
            OneHotEncoder(drop='first'),
            [ 'slope','thal']
        ),
         (
            'ordinal-cat_restecg',
            OrdinalEncoder(categories=[['lv hypertrophy', 'normal', 'st-t abnormality']]),
            ['restecg']
        ),
         (
            'ordinal-cat_cp',
            OrdinalEncoder(categories=[['typical angina', 'asymptomatic', 'non-anginal', 'atypical angina']]),
            ['cp']
        ),

         (
            'ordinal-cat_ca',
            OrdinalEncoder(categories=[['0', '1', '2','3']]),
            ['ca']
        ),

    ]
)

# Define model pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature',feature_selection),
    ('scaler',StandardScaler()),
    ('regressor', LinearRegression())
])

In [ ]:
# features = ["age", "is_male", "trestbps",  "thalch",  "oldpeak","is_heart_disease","chol","exang", "fbs", "thal", "cp", "restecg", "slope"]
X = df.dropna(subset=["chol"]).drop(columns=["chol"])
y = np.log(df.dropna(subset=["chol"])["chol"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

y_pred =np.exp(pipeline.predict(X_test))

mse = mean_squared_error(np.exp(y_test), y_pred)
r2 = r2_score(np.exp(y_test), y_pred)
mape = metrics.mean_absolute_percentage_error(np.exp(y_test), y_pred)

y_pred_train = np.exp(pipeline.predict(X_train))

mse_train = mean_squared_error(np.exp(y_train), y_pred_train)
r2_train = r2_score(np.exp(y_train), y_pred_train)
mape_train = metrics.mean_absolute_percentage_error(np.exp(y_train), y_pred_train)

metrics_dict = {
    'Test': [mse, mape, r2],
    'Train': [mse_train, mape_train, r2_train]
}



pd.DataFrame(metrics_dict, index=['MSE', 'MAPE', 'R2'])

/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:783: UserWarning:

k=20 is greater than n_features=17. All the features will be returned.



,Test,Train
MSE,2590.812279,3480.395100
MAPE,0.170077,0.173508
R2,0.043615,0.033004


In [ ]:
features = ["age", "is_male", "trestbps",  "thalch",  "oldpeak","is_heart_disease","ca","exang", "fbs", "thal", "cp", "restecg", "slope"]
missing_data = df[df["chol"].isna()]
df.loc[df["chol"].isna(), "chol"] = pipeline.predict(missing_data[features]).astype(int)

In [ ]:
df['chol'].min()

5.0

In [ ]:
df.isna().sum()

,0
age,0
cp,0
trestbps,0
chol,0
fbs,0
restecg,0
thalch,0
exang,0
oldpeak,0
slope,0


# Add bins categories

In [ ]:
bins = [0, 40, 60, np.inf]
labels = ['Young', 'Middle-Aged', 'Senior']
df['age_category'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

In [ ]:
bins = [0, 90, 120, np.inf]
labels = ['Low', 'Normal', 'High']
df['trestbps_category'] = pd.cut(df['trestbps'], bins=bins, labels=labels, right=False)

In [ ]:
bins = [0, 200, 240, np.inf]
labels = ['Low', 'Normal', 'High']
df['chol_category'] = pd.cut(df['chol'], bins=bins, labels=labels, right=False)

# Train Logistic Regration to predict heart Disease

In [ ]:
df

,age,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,is_heart_disease,is_male,age_category,trestbps_category,chol_category
0,63,typical angina,145.000000,233.0,True,lv hypertrophy,150.000000,False,2.300000,downsloping,0.0,fixed defect,False,True,Senior,High,Normal
1,67,asymptomatic,160.000000,286.0,False,lv hypertrophy,108.000000,True,1.500000,flat,3.0,normal,True,True,Senior,High,High
2,67,asymptomatic,120.000000,229.0,False,lv hypertrophy,129.000000,True,2.600000,flat,2.0,reversable defect,True,True,Senior,High,Normal
3,37,non-anginal,130.000000,250.0,False,normal,187.000000,False,3.500000,downsloping,0.0,normal,False,True,Young,High,High
4,41,atypical angina,130.000000,204.0,False,lv hypertrophy,172.000000,False,1.400000,upsloping,0.0,normal,False,False,Middle-Aged,High,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,54,asymptomatic,127.000000,333.0,True,st-t abnormality,154.000000,False,0.000000,no_data,0.0,no_data,True,False,Middle-Aged,High,High
916,62,typical angina,132.132404,139.0,False,st-t abnormality,137.545665,False,0.878788,no_data,0.0,no_data,False,True,Senior,High,Low
917,55,asymptomatic,122.000000,223.0,True,st-t abnormality,100.000000,False,0.000000,no_data,0.0,fixed defect,True,True,Middle-Aged,High,Normal
918,58,asymptomatic,132.132404,385.0,True,lv hypertrophy,137.545665,False,0.878788,no_data,0.0,no_data,False,True,Middle-Aged,High,High


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   age                920 non-null    int32   
 1   cp                 920 non-null    object  
 2   trestbps           920 non-null    float64 
 3   chol               920 non-null    float64 
 4   fbs                920 non-null    bool    
 5   restecg            920 non-null    object  
 6   thalch             920 non-null    float64 
 7   exang              920 non-null    bool    
 8   oldpeak            920 non-null    float64 
 9   slope              920 non-null    object  
 10  ca                 920 non-null    float64 
 11  thal               920 non-null    object  
 12  is_heart_disease   920 non-null    bool    
 13  is_male            920 non-null    bool    
 14  age_category       920 non-null    category
 15  trestbps_category  920 non-null    category
 16  chol_cat

In [ ]:
X = df.drop(columns=['is_heart_disease'])
y = df['is_heart_disease']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=101, stratify=y)

In [ ]:
# הגדרת תכונות קטגוריות ומספריות
categorical_features = ['cp', 'restecg', 'slope', 'thal','age_category','trestbps_category','chol_category']
numerical_features = df.select_dtypes(include=['number']).columns.tolist()

# טרנספורמציה עבור תכונות מספריות
num_transformers = Pipeline([
    ('scaler', StandardScaler())
])

# טרנספורמציה עבור תכונות קטגוריות
cat_transformer = OneHotEncoder(drop='first', handle_unknown='ignore')

# שילוב הכל ב-ColumnTransformer
preprocessor = ColumnTransformer([
    ('poly', PolynomialFeatures(degree=2, include_bias=False), numerical_features),
    ('cat', cat_transformer, categorical_features),
    ('num', num_transformers, numerical_features)
])

# בניית פייפליין מלא
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selection', SelectKBest(score_func=f_regression, k=50)),
    ('regressor', LogisticRegression(max_iter=1000, solver='liblinear'))
])

In [ ]:
model = pipeline.fit(X_train, y_train)

In [ ]:
model.predict(X_val)

array([ True, False, False,  True, False,  True,  True, False,  True,
       False,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True, False,  True,  True, False,  True, False, False,
        True,  True, False,  True,  True,  True, False, False,  True,
        True,  True, False, False,  True, False,  True,  True, False,
        True, False, False,  True, False,  True, False,  True,  True,
        True, False, False,  True, False,  True,  True, False,  True,
       False, False,  True,  True,  True,  True, False, False, False,
        True, False,  True,  True,  True,  True,  True, False, False,
        True,  True,  True, False,  True,  True, False, False, False,
        True,  True,  True, False, False,  True,  True,  True,  True,
       False, False,  True, False,  True,  True,  True, False,  True,
        True, False,  True,  True,  True, False,  True,  True,  True,
        True, False, False, False, False,  True, False,  True, False,
        True, False,

In [ ]:
model.predict_proba(X_val)

array([[0.0564619 , 0.9435381 ],
       [0.94122053, 0.05877947],
       [0.6145926 , 0.3854074 ],
       [0.42645668, 0.57354332],
       [0.7892493 , 0.2107507 ],
       [0.16918889, 0.83081111],
       [0.02903342, 0.97096658],
       [0.79704008, 0.20295992],
       [0.20515445, 0.79484555],
       [0.88907921, 0.11092079],
       [0.04623274, 0.95376726],
       [0.16606524, 0.83393476],
       [0.02877538, 0.97122462],
       [0.91347422, 0.08652578],
       [0.1094315 , 0.8905685 ],
       [0.23015967, 0.76984033],
       [0.16518982, 0.83481018],
       [0.02937014, 0.97062986],
       [0.22329343, 0.77670657],
       [0.15078742, 0.84921258],
       [0.86934896, 0.13065104],
       [0.11977323, 0.88022677],
       [0.05202255, 0.94797745],
       [0.6456345 , 0.3543655 ],
       [0.07003272, 0.92996728],
       [0.73831388, 0.26168612],
       [0.87376563, 0.12623437],
       [0.20369347, 0.79630653],
       [0.17247821, 0.82752179],
       [0.93655329, 0.06344671],
       [0.

In [ ]:
metrics.confusion_matrix(y_val, model.predict(X_val))

array([[48, 14],
       [ 8, 68]])

In [ ]:
pd.DataFrame(metrics.confusion_matrix(y_val, model.predict(X_val),normalize='true'),
             columns=['Predicted Heart Disease (0)', 'Predicted Heart Disease (1)'],
             index=['Actual Heart Disease (0)', 'Actual Heart Disease (1)'])

,Predicted Heart Disease (0),Predicted Heart Disease (1)
Actual Heart Disease (0),0.774194,0.225806
Actual Heart Disease (1),0.105263,0.894737


In [ ]:
preds = model.predict_proba(X_val)[:,1] >= 0.50
metrics.confusion_matrix(y_val, preds)

array([[48, 14],
       [ 8, 68]])

In [ ]:
probabilities = model.predict_proba(X_val)[:,1]
fpr, tpr, thresholds = roc_curve(y_val, probabilities)
roc_auc = auc(fpr, tpr)

In [ ]:
# prompt: display roc curve using plotly with threshold values as tooltips

fig = go.Figure()

# Add the ROC curve
fig.add_trace(go.Scatter(
    x=fpr,
    y=tpr,
    mode='lines',
    name=f'ROC Curve (AUC = {roc_auc:.2f})',
    hovertemplate = 'False Positive Rate: %{x:.3f}<br>True Positive Rate: %{y:.3f}<br>Threshold: %{text:.3f}<extra></extra>',
    text = thresholds
))

# Add a diagonal line for random guessing
fig.add_trace(go.Scatter(
    x=[0, 1],
    y=[0, 1],
    mode='lines',
    line=dict(dash='dash'),
    name='Random Guessing'
))


fig.update_layout(
    title='Receiver Operating Characteristic (ROC) Curve',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    width=800,
    height=600
)

fig.show()


In [ ]:
youden_index = tpr - fpr
optimal_threshold_index = youden_index.argmax()
optimal_threshold = thresholds[optimal_threshold_index]

print(f"Optimal threshold (Youden's Index): {optimal_threshold}")

Optimal threshold (Youden's Index): 0.4934966438951617


In [ ]:
f1_scores = [f1_score(y_val, (probabilities >= threshold).astype(int)) for threshold in thresholds]
optimal_threshold_f1 = thresholds[f1_scores.index(max(f1_scores))]
print(f"Optimal threshold (F1 score): {optimal_threshold_f1}")


Optimal threshold (F1 score): 0.4934966438951617


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=fpr,
    y=tpr,
    mode='lines',
    name=f'ROC Curve (AUC = {roc_auc:.2f})',
    hovertemplate='False Positive Rate: %{x:.3f}<br>True Positive Rate: %{y:.3f}<br>Threshold: %{text:.3f}<extra></extra>',
    text=thresholds
))

# Add a diagonal line for random guessing
fig.add_trace(go.Scatter(
    x=[0, 1],
    y=[0, 1],
    mode='lines',
    line=dict(dash='dash'),
    name='Random Guessing'
))

# Highlight the Optimal Threshold
fig.add_trace(go.Scatter(
    x=[fpr[optimal_threshold_index]],
    y=[tpr[optimal_threshold_index]],
    mode='markers',
    marker=dict(size=10, color='red', symbol='star'),
    name=f'Optimal Threshold: {optimal_threshold:.3f}'
))

# Add annotation for optimal threshold
fig.add_annotation(
    x=fpr[optimal_threshold_index],
    y=tpr[optimal_threshold_index],
    text=f'Optimal Threshold: {optimal_threshold:.3f}',
    showarrow=True,
    arrowhead=2,
    ax=-40,
    ay=-40,
    bgcolor="white",
    bordercolor="black"
)

# Update layout
fig.update_layout(
    title='Receiver Operating Characteristic (ROC) Curve',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    width=800,
    height=600
)

# Show the figure
fig.show()

In [ ]:
y_pred_optimal = (probabilities >= optimal_threshold).astype(int)
print(classification_report(y_val, y_pred_optimal))

              precision    recall  f1-score   support

       False       0.87      0.77      0.82        62
        True       0.83      0.91      0.87        76

    accuracy                           0.85       138
   macro avg       0.85      0.84      0.84       138
weighted avg       0.85      0.85      0.85       138



# Model Interpretation

In [ ]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('poly',
                                                  PolynomialFeatures(include_bias=False),
                                                  ['age', 'trestbps', 'chol',
                                                   'thalch', 'oldpeak', 'ca']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['cp', 'restecg', 'slope',
                                                   'thal', 'age_category',
                                                   'trestbps_category',
                                                   'chol_category']),
                                                 ('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'trestbps', 'chol',
                                                   'thalch', 'oldpeak',
                                                   'ca'])])),
                ('feature_selection',
                 SelectKBest(k=50,
                             score_func=<function f_regression at 0x7e3b84b616c0>)),
                ('regressor',
                 LogisticRegression(max_iter=1000, solver='liblinear'))])

In [ ]:
columns = pipeline[:1].get_feature_names_out()
columns

array(['poly__age', 'poly__trestbps', 'poly__chol', 'poly__thalch',
       'poly__oldpeak', 'poly__ca', 'poly__age^2', 'poly__age trestbps',
       'poly__age chol', 'poly__age thalch', 'poly__age oldpeak',
       'poly__age ca', 'poly__trestbps^2', 'poly__trestbps chol',
       'poly__trestbps thalch', 'poly__trestbps oldpeak',
       'poly__trestbps ca', 'poly__chol^2', 'poly__chol thalch',
       'poly__chol oldpeak', 'poly__chol ca', 'poly__thalch^2',
       'poly__thalch oldpeak', 'poly__thalch ca', 'poly__oldpeak^2',
       'poly__oldpeak ca', 'poly__ca^2', 'cat__cp_atypical angina',
       'cat__cp_non-anginal', 'cat__cp_typical angina',
       'cat__restecg_normal', 'cat__restecg_st-t abnormality',
       'cat__slope_flat', 'cat__slope_no_data', 'cat__slope_upsloping',
       'cat__thal_no_data', 'cat__thal_normal',
       'cat__thal_reversable defect', 'cat__age_category_Senior',
       'cat__age_category_Young', 'cat__trestbps_category_Low',
       'cat__trestbps_category_Nor

In [ ]:
coefs = pd.DataFrame(pipeline[-1].coef_, columns=columns).transpose().rename(columns={0:'log-odds'})
coefs

,log-odds
poly__age,-0.075041
poly__trestbps,-0.037452
poly__chol,-0.028354
poly__thalch,0.115006
poly__oldpeak,0.236628
poly__ca,0.035092
poly__age^2,0.000012
poly__age trestbps,0.001218
poly__age chol,0.000013
poly__age thalch,-0.000412


In [ ]:
coefs['odds'] = np.exp(coefs['log-odds'])
coefs

,log-odds,odds
poly__age,-0.075041,0.927705
poly__trestbps,-0.037452,0.963241
poly__chol,-0.028354,0.972044
poly__thalch,0.115006,1.121880
poly__oldpeak,0.236628,1.266969
poly__ca,0.035092,1.035715
poly__age^2,0.000012,1.000012
poly__age trestbps,0.001218,1.001218
poly__age chol,0.000013,1.000013
poly__age thalch,-0.000412,0.999589


# Predict

In [ ]:
import joblib
joblib.dump(pipeline, 'model_heart_disease.pkl')

['model_heart_disease.pkl']

In [ ]:
def predict_heart_disease(age, cp, trestbps, chol, fbs, restecg, thalch, exang, oldpeak, slope, ca, thal, is_male,age_category,chol_category,trestbps_category):
    """
    טוען את המודל ומשתמש בנתונים כדי לבצע חיזוי של מחלת לב.

    פרמטרים:
    age (int): גיל
    cp (str): סוג כאב חזה ('asymptomatic', 'typical angina' וכו')
    trestbps (float): לחץ דם במנוחה
    chol (float): רמת כולסטרול
    fbs (bool): האם רמת הסוכר בצום גבוהה מ-120? (True/False)
    restecg (str): תוצאות ECG במנוחה ('normal', 'st-t abnormality' וכו')
    thalch (float): דופק מרבי
    exang (bool): האם מופיעה אנגינה בזמן מאמץ? (True/False)
    oldpeak (float): דיכוי ST ביחס למנוחה
    slope (str): שיפוע מקטע ST ('upsloping', 'flat', 'downsloping')
    ca (float): מספר כלי דם מוצרים (0-3, יכול להיות NaN)
    thal (str): סוג תליום ('normal', 'fixed defect', 'reversable defect')
    is_male (bool): האם המטופל גבר? (True/False)
    age_category: 0-40 Young,40-60 Middle-Aged,60+ Senior
    chol_category: 0-200 Low, 200-240 Normal, 240+ High
    trestbps_category: trestbps <90 Low, 90 <= trestbps  <120 Normal ,trestbps >= High
    מחזיר:
    int: 1 (חולה במחלת לב) או 0 (לא חולה)
    """

    # טעינת המודל
    loaded_model = joblib.load('model_heart_disease.pkl')

    # יצירת DataFrame עם הנתון
    data = pd.DataFrame([{
        'age': age,
        'cp': cp,
        'trestbps': trestbps,
        'chol': chol,
        'fbs': fbs,
        'restecg': restecg,
        'thalch': thalch,
        'exang': exang,
        'oldpeak': oldpeak,
        'slope': slope,
        'ca': ca,
        'thal': thal,
        'is_male': is_male,
        'age_category':age_category,
        'chol_category':chol_category,
        'trestbps_category':trestbps_category
    }])

    # ביצוע חיזוי
    prediction = loaded_model.predict(data)

    return prediction[0]  # מחזיר את התוצאה (0 = לא חולה, 1 = חולה)

In [ ]:
result = predict_heart_disease(
    age=54,
    cp='asymptomatic',
    trestbps=130.0,
    chol=110.0,
    fbs=False,
    restecg='normal',
    thalch=130.0,
    exang=False,
    oldpeak=1.0,
    slope='flat',
    ca=0,
    thal='normal',
    is_male=True,
    age_category='Middle-Aged',
    chol_category='Low',
    trestbps_category='High'
)

print("Prediction:", "Heart Disease" if result == 1 else "No Heart Disease")

Prediction: Heart Disease


In [ ]:
result = predict_heart_disease(
    age=54,
    cp='asymptomatic',
    trestbps=80.0,
    chol=219.0,
    fbs=False,
    restecg='st-t abnormality',
    thalch=130.0,
    exang=True,
    oldpeak=1.0,
    slope='flat',
    ca=2,
    thal='reversable defect',
    is_male=True,
    age_category='Middle-Aged',
    chol_category='Normal',
    trestbps_category='Low'
)

print("Prediction:", "Heart Disease" if result == 1 else "No Heart Disease")

Prediction: Heart Disease


In [ ]:
result = predict_heart_disease(
    age=30,
    cp='asymptomatic',
    trestbps=125.0,
    chol=125.0,
    fbs=False,
    restecg='normal',
    thalch=120.0,
    exang=False,
    oldpeak=0.0,
    slope='downsloping',
    ca=0,
    thal='normal',
    is_male=True,
    age_category='Young',
    chol_category='Low',
    trestbps_category='Low')

print("Prediction:", "Heart Disease" if result == 1 else "No Heart Disease")

Prediction: No Heart Disease
